# Imagen: Dogs vs Cats dataset

In [2]:
import sys
!{sys.executable} -m pip install tensorflow --quiet



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
import os, zipfile
import tensorflow as tf
import pickle


In [4]:
import zipfile
import pathlib

zip_path = "deep-learning.zip"
extract_path = "deep-learning"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

BASE_DIR   = pathlib.Path("deep-learning/deep-learning")
IMG_SIZE   = (224, 224)
BATCH_SIZE = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
              BASE_DIR / "train",
              validation_split=0.2, subset="training", seed=123,
              image_size=IMG_SIZE, batch_size=BATCH_SIZE,
              label_mode="binary")

val_ds   = tf.keras.preprocessing.image_dataset_from_directory(
              BASE_DIR / "train",
              validation_split=0.2, subset="validation", seed=123,
              image_size=IMG_SIZE, batch_size=BATCH_SIZE,
              label_mode="binary")

test_ds  = tf.keras.preprocessing.image_dataset_from_directory(
              BASE_DIR / "test",
              shuffle=False,
              image_size=IMG_SIZE, batch_size=BATCH_SIZE,
              label_mode="binary")

print("Clases detectadas:", train_ds.class_names)


Found 557 files belonging to 2 classes.
Using 446 files for training.
Found 557 files belonging to 2 classes.
Using 111 files for validation.
Found 140 files belonging to 2 classes.
Clases detectadas: ['cats', 'dogs']


In [5]:
!pip install tensorflow.keras

In [9]:
from tensorflow import keras
from tensorflow.keras import layers


image_model = keras.Sequential([
    layers.Rescaling(1./255, input_shape=(224, 224, 3)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

image_model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


image_model.fit(train_ds,
          epochs=15,
          validation_data=val_ds)

test_loss, test_acc = image_model.evaluate(test_ds)
print("Test accuracy:", test_acc)


Epoch 1/15


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - accuracy: 0.5080 - loss: 40.8974 - val_accuracy: 0.5135 - val_loss: 4.6592
Epoch 2/15
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - accuracy: 0.5688 - loss: 2.0141 - val_accuracy: 0.4955 - val_loss: 0.8062
Epoch 3/15
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - accuracy: 0.6334 - loss: 0.6859 - val_accuracy: 0.4865 - val_loss: 1.1659
Epoch 4/15
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step - accuracy: 0.5783 - loss: 0.8428 - val_accuracy: 0.5045 - val_loss: 1.3156
Epoch 5/15
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - accuracy: 0.5370 - loss: 1.1940 - val_accuracy: 0.5676 - val_loss: 1.0013
Epoch 6/15
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - accuracy: 0.6732 - loss: 0.6400 - val_accuracy: 0.6306 - val_loss: 0.7157
Epoch 7/15
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step - accuracy: 0.7651 - loss: 0.4840 - val_accuracy: 0.5676 - val_loss: 0.7299
Epoch 8/15
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step - accuracy: 0.7512 - loss: 0.5014 - val_accuracy: 0.5766 - val_loss: 0

In [7]:
import sys
!{sys.executable} -m pip install scikit-learn --quiet


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [10]:
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import numpy as np
y_true = np.concatenate([y for x, y in test_ds], axis=0)

y_pred_prob = image_model.predict(test_ds)
y_pred = (y_pred_prob > 0.5).astype("int32").flatten()
print(classification_report(y_true, y_pred, target_names=test_ds.class_names))


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
              precision    recall  f1-score   support

        cats       0.55      0.76      0.64        70
        dogs       0.61      0.39      0.47        70

    accuracy                           0.57       140
   macro avg       0.58      0.57      0.56       140
weighted avg       0.58      0.57      0.56       140



2025-05-26 22:15:19.871483: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [12]:
image_model.save("image_model.keras")


# Text SMS Spam Collection

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

In [14]:
!{sys.executable} -m pip install pandas --quiet


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [33]:
import pandas as pd
df = pd.read_csv("spam.csv", encoding="latin-1")[['v1', 'v2']]
df = df.rename(columns={'v1': 'label', 'v2': 'text'}).copy()
text_model = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english')),
    ('clf', MultinomialNB())
])

In [34]:
df.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [16]:

df['label'] = df['label'].str.strip().map({'ham': 0, 'spam': 1}).astype('int32')
df['text']  = df['text'].astype(str)

In [19]:
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import GlobalAveragePooling1D, Dense


X_train, X_test, y_train, y_test = train_test_split(
    df['text'], df['label'], test_size=0.2, random_state=42)

MAX_TOKENS = 10000
MAX_LEN = 100
EMBED_DIM = 16

vectorizer = TextVectorization(max_tokens=MAX_TOKENS,
                               output_sequence_length=MAX_LEN)
vectorizer.adapt(X_train)

text_model = Sequential([
    vectorizer,
    Embedding(MAX_TOKENS, EMBED_DIM),
    GlobalAveragePooling1D(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

text_model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

X_train_np = X_train.to_numpy(dtype=object)
X_test_np  = X_test.to_numpy(dtype=object)

text_model.fit(X_train_np, y_train.to_numpy(),
          epochs=15, batch_size=32,
          validation_split=0.1)

loss, acc = text_model.evaluate(X_test_np, y_test.to_numpy())
print(f"Test accuracy: {acc:.3f}")

Epoch 1/15
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8452 - loss: 0.4377 - val_accuracy: 0.8565 - val_loss: 0.3892
Epoch 2/15
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8777 - loss: 0.3471 - val_accuracy: 0.8565 - val_loss: 0.3812
Epoch 3/15
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8708 - loss: 0.3482 - val_accuracy: 0.8565 - val_loss: 0.3646
Epoch 4/15
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8675 - loss: 0.3295 - val_accuracy: 0.8565 - val_loss: 0.3244
Epoch 5/15
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8630 - loss: 0.2700 - val_accuracy: 0.9350 - val_loss: 0.1973
Epoch 6/15
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9486 - loss: 0.1536 - val_accuracy: 0.9641 - val_loss: 0.1341
Epoch 7/15
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9720 - loss: 0.0987 - val_accuracy: 0.9574 - val_loss: 0.1177
Epoch 8/15
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9819 - loss: 0.0607 - val_accuracy: 0.

In [20]:
from sklearn.metrics import classification_report

y_pred_probs = text_model.predict(X_test_np)

y_pred_labels = (y_pred_probs > 0.5).astype(int)

print(classification_report(y_test, y_pred_labels,
                            target_names=['ham', 'spam']))


35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
              precision    recall  f1-score   support

         ham       0.98      1.00      0.99       965
        spam       0.98      0.88      0.93       150

    accuracy                           0.98      1115
   macro avg       0.98      0.94      0.96      1115
weighted avg       0.98      0.98      0.98      1115



In [21]:
text_model.save("text_model.keras")

# Regression: House Prices

In [23]:
train = pd.read_csv("train.csv")

In [24]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [25]:
len(train)

1460

In [26]:
len(train.columns)

81

In [27]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from tensorflow.keras import layers, Sequential
import numpy as np

train = train.fillna(train.mean(numeric_only=True))

y = train.pop('SalePrice')
X = train.copy()

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

cat_cols = X_train.select_dtypes(include=['object', 'category']).columns.tolist()
num_cols = X_train.select_dtypes(exclude=['object', 'category']).columns.tolist()

pre = ColumnTransformer([
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols),
        ('num', MinMaxScaler(), num_cols)
    ])

X_train_prep = pre.fit_transform(X_train)
X_test_prep  = pre.transform(X_test)

 
y_scaler = MinMaxScaler()
y_train_scaled = y_scaler.fit_transform(y_train.values.reshape(-1,1))

input_dim = X_train_prep.shape[1]

regression_model = Sequential([
    layers.Dense(128, activation='relu', input_shape=(input_dim,)),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])

regression_model.compile(
    optimizer='adam',
    loss='mean_squared_error',
    metrics=['mean_absolute_error']
)

history = regression_model.fit(
    X_train_prep, y_train_scaled,
    epochs=50, batch_size=32,
    validation_split=0.1,
    verbose=1
)

y_pred_scaled = regression_model.predict(X_test_prep)
y_pred = y_scaler.inverse_transform(y_pred_scaled)

mae = np.mean(np.abs(y_test.values.reshape(-1,1) - y_pred))
print(f"Test MAE (original scale): {mae:.2f}")


Epoch 1/50


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0348 - mean_absolute_error: 0.1405 - val_loss: 0.0053 - val_mean_absolute_error: 0.0585
Epoch 2/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0049 - mean_absolute_error: 0.0533 - val_loss: 0.0039 - val_mean_absolute_error: 0.0492
Epoch 3/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0024 - mean_absolute_error: 0.0375 - val_loss: 0.0037 - val_mean_absolute_error: 0.0460
Epoch 4/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0022 - mean_absolute_error: 0.0341 - val_loss: 0.0034 - val_mean_absolute_error: 0.0450
Epoch 5/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0015 - mean_absolute_error: 0.0273 - val_loss: 0.0036 - val_mean_absolute_error: 0.0473
Epoch 6/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0011 - mean_absolute_error: 0.0244 - val_loss: 0.0033 - val_mean_absolute_error: 0.0443
Epoch 7/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8.4624e-04 - mean_absolute_error: 0.0216 - val_loss: 0.0032 - val

In [28]:
loss, mae = regression_model.evaluate(X_test_prep, y_test)
print(f"Test MAE: {mae:.0f}")


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 38920499200.0000 - mean_absolute_error: 178000.3125 
Test MAE: 178840


In [29]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import math
y_pred_scaled = regression_model.predict(X_test_prep).squeeze()

y_pred = y_scaler.inverse_transform(y_pred_scaled.reshape(-1, 1)).squeeze()

mae  = mean_absolute_error(y_test, y_pred)
rmse = math.sqrt(mean_squared_error(y_test, y_pred))
r2   = r2_score(y_test, y_pred)

print(f"MAE:  ${mae:,.0f}")
print(f"RMSE: ${rmse:,.0f}")
print(f"R² score: {r2:.3f}")


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
MAE:  $26,211
RMSE: $40,483
R² score: 0.786


In [30]:
regression_model.save("regression_model.keras")

In [32]:

with open("preprocessor.pkl", "wb") as f:
    pickle.dump(pre, f)

with open("y_scaler.pkl", "wb") as f:
    pickle.dump(y_scaler, f)


In [31]:
!pip freeze > requirements.txt
